In [83]:
import pandas as pd
from datetime import datetime
import os
from time import time
import glob

In [84]:
start = time()

In [85]:
def reduce_rides_data(files, tag_column, tag_name, file_reduced_folder):
    
    for filename in files:

        dirname, fname = os.path.split(filename)
        
        df = pd.read_csv(filename)
        tag_value = df[tag_column].iloc[0]

        file_tag = tag_value
        if isinstance(file_tag, str):
            if len(file_tag) == 10:  # only date
                file_tag = datetime.strptime(file_tag, "%Y-%m-%d").strftime("%Y%m%d_%H")
            else:
                file_tag = datetime.strptime(file_tag, "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d_%H")

        reduced = dict(
            tag = tag_name,
            tag_value = tag_value,
            
            rides_count = len(df),

            passenger_count_mean = df["passenger_count"].mean(),
            passenger_count_median = df["passenger_count"].median(),

            trip_distance_sum = df["trip_distance"].sum(),
            trip_distance_mean = df["trip_distance"].mean(),
            trip_distance_median = df["trip_distance"].median(),

            tip_amount_sum = df["tip_amount"].sum(),
            tip_amount_mean = df["tip_amount"].mean(),
            tip_amount_median = df["tip_amount"].median(),

            humidity_mean = df["humidity"].mean(),
            humidity_median = df["humidity"].median(),

            pressure_mean = df["pressure"].mean(),
            pressure_median = df["pressure"].median(),

            wind_speed_mean = df["wind_speed"].mean(),
            wind_speed_median = df["wind_speed"].median()
        )

        reduced_filename = f"{file_reduced_folder}/{fname}"
        df_reduced = pd.DataFrame([reduced])
        df_reduced.to_csv(reduced_filename, index=False)

In [86]:
def merge_files(files, dest_filename):
    for file in files:
        df = pd.read_csv(file)
        
        if os.path.exists(dest_filename):
            df.to_csv(dest_filename, mode='a', header=False, index=False)
        else:
            df.to_csv(dest_filename, index=False)

        os.remove(file)

Reduce mapped files

In [87]:
files_by_days = glob.glob("./data/processing/days*.csv")
files_by_hours = glob.glob("./data/processing/hours*.csv")
files_by_weekdays = glob.glob("./data/processing/weekdays*.csv")
files_by_weather = glob.glob("./data/processing/weather*.csv")

file_reduced_folder = "./data/reduced/"

In [88]:
reduce_rides_data(files_by_days, "by_day", "days", file_reduced_folder)
reduce_rides_data(files_by_hours, "by_hour", "hours", file_reduced_folder)
reduce_rides_data(files_by_weekdays, "by_weekday", "weekdays", file_reduced_folder)
reduce_rides_data(files_by_weather, "weather_description_code", "weather", file_reduced_folder)

Merge reduced files

In [89]:
files_by_days = glob.glob("./data/reduced/days*.csv")
files_by_hours = glob.glob("./data/reduced/hours*.csv")
files_by_weekdays = glob.glob("./data/reduced/weekdays*.csv")
files_by_weather = glob.glob("./data/reduced/weather*.csv")

file_merged_template = "./data/reduced/{0}.csv"

In [90]:
merge_files(files_by_days, "./data/reduced/days.csv")
merge_files(files_by_hours, "./data/reduced/hours.csv")
merge_files(files_by_weekdays, "./data/reduced/weekdays.csv")
merge_files(files_by_weather, "./data/reduced/weather.csv")

In [91]:
print(f"Done in {time()-start} sec")

Done in 163.66869640350342 sec
